<a href="https://colab.research.google.com/github/uri-rizo2/Machine-Learning-CSCI191t/blob/main/MLP_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uriel Amezcua

11/14/21

csci 191t


# Backpropagation for training a multilayer perceptron 

In [22]:
#imports 
from math import exp
from random import seed
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import numpy as np
import math
import plotly.express as px

In [42]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network


In [29]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation


In [4]:
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

In [55]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

In [56]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

In [7]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [8]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

In [9]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [17]:
 
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [52]:
# Test training backprop algorithm using given dataset 
seed(1)
dataset = [[0,0,0],
	[0,1,1],
	[1,0,0],
	[1,1,0],]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=2.416
>epoch=1, lrate=0.500, error=2.150
>epoch=2, lrate=0.500, error=1.943
>epoch=3, lrate=0.500, error=1.807
>epoch=4, lrate=0.500, error=1.725
>epoch=5, lrate=0.500, error=1.675
>epoch=6, lrate=0.500, error=1.645
>epoch=7, lrate=0.500, error=1.626
>epoch=8, lrate=0.500, error=1.613
>epoch=9, lrate=0.500, error=1.604
>epoch=10, lrate=0.500, error=1.598
>epoch=11, lrate=0.500, error=1.593
>epoch=12, lrate=0.500, error=1.589
>epoch=13, lrate=0.500, error=1.586
>epoch=14, lrate=0.500, error=1.584
>epoch=15, lrate=0.500, error=1.582
>epoch=16, lrate=0.500, error=1.580
>epoch=17, lrate=0.500, error=1.578
>epoch=18, lrate=0.500, error=1.577
>epoch=19, lrate=0.500, error=1.575
[{'weights': [0.3351548675385257, 0.6893833504154762, 0.8794546808781708], 'output': 0.8692662362011159, 'delta': -0.006337578336279663}, {'weights': [0.2705940285665871, 0.3449306499559463, 0.32896353694842995], 'output': 0.7186297253118313, 'delta': -0.00454232099977171}]
[{'weights': [0

In [19]:
#predictions vs. actual 

for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=1
Expected=1, Got=0
Expected=1, Got=1
Expected=0, Got=0
